In [1]:
# DisulfideBond Playground
# Playing with the DisulfideBond class
# Author: Eric G. Suchanek, PhD.
# (c) 2022 Eric G. Suchanek, PhD., All Rights Reserved
# License: BSD
# Last Modification: 12/12/22
# Cα Cβ Sγ

# important preamble

import pandas as pd

import pyvista as pv
from pyvista import set_plot_theme

from Bio.PDB import *

# for using from the repo we 
from proteusPy import *
from proteusPy.proteusGlobals import *
from proteusPy.Disulfide import *


# override any default PDB globals
# location for PDB repository
PDB_ROOT = '/Users/egs/PDB/'

# location of cleaned PDB files - these are not stored in the repo
PDB_GOOD = '/Users/egs/PDB/good/'

# from within the repo 
PDB_REPO = '../pdb/'

# location of the compressed Disulfide .pkl files
MODELS = f'{PDB_ROOT}models/'

# pyvista setup for notebooks
pv.set_jupyter_backend('ipyvtklink')
set_plot_theme('dark')


In [2]:
# Comment these out since they take so long.
# DownloadDisulfides(pdb_home=PDB_ORIG, model_home=MODELS, reset=False)

#ExtractDisulfides(numb=1000, pdbdir=PDB_GOOD, modeldir=MODELS, verbose=False, quiet=False)

PDB_SS = None
PDB_SS = DisulfideLoader(verbose=True, modeldir=MODELS)

ss_list = DisulfideList([], 'tmp')


Reading disulfides from: /Users/egs/PDB/models/PDB_all_ss.pkl
Disulfides Read: 16222
Reading disulfide dict from: /Users/egs/PDB/models/PDB_all_ss_dict.pkl
Reading Torsion DF /Users/egs/PDB/models/PDB_all_SS_torsions.csv.
Read torsions DF.
PDB IDs parsed: 2000
Total Space Used: 4035518 bytes.


In [3]:

# one disulfide from the database
ss = Disulfide()
ss = PDB_SS[0]
print(f'SS: {ss}')

# get all disulfides for one structure. Make a 
# DisulfideList object to hold it
ss4yys = DisulfideList([], '4yys')
ss4yys = PDB_SS['4yys']

#ss4crn = DisulfideList([], '1crn')
#ss4crn = PDB_SS['1crn']

tot_ss = len(ss4yys) # number off ssbonds
print(f'tot {tot_ss}')


SS: <Disulfide 4yys_22A_65A SourceID: 4yys Proximal: 22 A Distal: 65 A>
tot 6


In [ ]:
ss6fuf = PDB_SS['6fuf']


In [5]:
tors = pd.DataFrame()
tors = PDB_SS.getTorsions('4yys')
tors

,source,ss_id,proximal,distal,chi1,chi2,chi3,chi4,chi5,energy,ca_distance,phi_prox,psi_prox,phi_dist,psi_dist
0,4yys,4yys_22A_65A,22,65,174.629233,82.517710,-83.322249,-62.523644,-73.827286,1.696237,4.502086,-149.776903,128.686790,-105.731172,16.431694
1,4yys,4yys_56A_98A,56,98,-50.239063,-85.583916,97.275447,70.535692,179.046592,2.112566,4.967417,-113.982916,-30.489936,-59.378573,125.462589
2,4yys,4yys_156A_207A,156,207,62.598713,172.940042,-95.352637,-23.070934,-55.158480,2.331733,5.292317,-156.930659,150.741801,60.457994,22.170381
3,4yys,4yys_22B_65B,22,65,173.666078,88.297996,-82.387276,-65.997032,-72.289506,1.958823,4.532387,-154.247515,130.848015,-103.903213,16.298008
4,4yys,4yys_56B_98B,56,98,-56.410909,-81.401941,94.310784,67.035993,178.852441,1.444608,4.768629,-113.329915,-30.319989,-53.571262,120.853837
5,4yys,4yys_156B_207B,156,207,66.747274,168.512311,-98.492801,-20.451335,-57.455022,2.806933,5.393519,-160.664736,149.278415,58.933546,21.029930


In [6]:
check_chains('4yys', PDB_GOOD)

ssbond dict: {1: ('22', '65', 'A', 'A'), 2: ('56', '98', 'A', 'A'), 3: ('156', '207', 'A', 'A'), 4: ('22', '65', 'B', 'B'), 5: ('56', '98', 'B', 'B'), 6: ('156', '207', 'B', 'B')}
multiple chains. [<Chain id=A>, <Chain id=B>]
Chain: A, length: 504
Chain: B, length: 454
chain lengths are unequal: [504, 454]


False

In [7]:
ss4yys_a = ss4yys.by_chain('A')
ss4yys_a


DisulfideList([<Disulfide 4yys_22A_65A SourceID: 4yys Proximal: 22 A Distal: 65 A>,
               <Disulfide 4yys_56A_98A SourceID: 4yys Proximal: 56 A Distal: 98 A>,
               <Disulfide 4yys_156A_207A SourceID: 4yys Proximal: 156 A Distal: 207 A>])

In [8]:
ss4yys_b = ss4yys.by_chain('B')
ss4yys_b

DisulfideList([<Disulfide 4yys_22B_65B SourceID: 4yys Proximal: 22 B Distal: 65 B>,
               <Disulfide 4yys_56B_98B SourceID: 4yys Proximal: 56 B Distal: 98 B>,
               <Disulfide 4yys_156B_207B SourceID: 4yys Proximal: 156 B Distal: 207 B>])

In [9]:
ss4yys_a1 = ss4yys_a[0]
print(ss4yys_a1.repr_ss_coords())



Proximal Coordinates:
   N: <Vector -2.36, -20.48, 5.21>
   Cα: <Vector -2.10, -19.89, 3.90>
   C: <Vector -1.12, -18.78, 4.12>
   O: <Vector -1.30, -17.96, 5.03>
   Cβ: <Vector -3.38, -19.31, 3.32>
   Sγ: <Vector -3.24, -18.40, 1.76>
   Cprev <Vector -2.67, -21.75, 5.36>
   Nnext: <Vector -0.02, -18.76, 3.36>
 Distal Coordinates:
   N: <Vector -0.60, -18.71, -1.62>
   Cα: <Vector -0.48, -19.10, -0.22>
   C: <Vector 0.92, -19.52, 0.18>
   O: <Vector 1.10, -20.09, 1.25>
   Cβ: <Vector -1.48, -20.23, 0.08>
   Sγ: <Vector -3.22, -19.69, 0.18>
   Cprev <Vector -0.73, -17.44, -2.01>
   Nnext: <Vector 1.92, -19.18, -0.63>




In [10]:
ss4yys_b1 = ss4yys_b[0]
ss4yys_b1

<Disulfide 4yys_22B_65B SourceID: 4yys Proximal: 22 B Distal: 65 B>

In [11]:
Distance_RMS(ss4yys_a1, ss4yys_b1)

0.1145026461405659

In [12]:
Torsion_RMS(ss4yys_a1, ss4yys_b1)

7.045760800877229

In [13]:
ss4yys_a1 == ss4yys_b1

False

In [14]:
chns = ss4yys.get_chains()
ss4yys.has_chain('yyy')
chns

{'A', 'B'}

In [ ]:
# load SS bonds by PDB ID
ss1 = PDB_SS['4yys']
print(ss1)
print(ss1[0].get_full_id())


In [ ]:
# you can loop over the IDList list and extract by ID
#
for id in PDB_SS.IDList[:2]:    # just show last 5
    # get the SS bonds for the given ID
    ssb = PDB_SS[id]
    numb_ss = len(ssb)
    print(f'ID: {id} has {numb_ss} Disulfides:')
    for bond in ssb:
        print(bond)
    print('\n')
    

In [ ]:
ss_list = PDB_SS.getlist()
ss0 = ss_list[0]

print(ss0.proximal_residue_fullid)
print(ss0.chi3)
len(ss_list)


In [ ]:
# routine creates 2 lists  for left-handed and right-handed disulfides 
ss_list = PDB_SS.getlist()
left_handed = DisulfideList([], 'left_handed')
right_handed = DisulfideList([], 'right_handed')

i = 0

for i in range(0, len(ss_list)):
    ss = ss_list[i]
    if ss.chi3 < 0:
        left_handed.append(ss)
    else:
        right_handed.append(ss)


print(f'Left Handed: {len(left_handed)}, Right Handed: {len(right_handed)}')



In [ ]:
# Make a plot

import pyvista as pv
from pyvista import set_plot_theme

pv.set_jupyter_backend('ipyvtklink')
set_plot_theme('document')

from proteusPy.proteusGlobals import ATOM_RADII_COVALENT, ATOM_RADII_CPK, ATOM_COLORS
ss1 = PDB_SS[0]

coords = ss1.internal_coords()
atoms = ('N', 'C', 'C', 'O', 'C', 'SG', 'N', 'C', 'C', 'O', 'C', 'SG')

i = 0
p = pv.Plotter()

for atom in atoms:
    rad = ATOM_RADII_CPK[atom]
    p.add_mesh(pv.Sphere(center=coords[i], radius=rad),)
    print(f'Atom: {atom} Coords: {coords[i]} rad: {rad}')
    i += 1

p.show()


In [ ]:
p = pv.Plotter()
p.add_mesh(pv.Sphere(center=[0,0,0], radius=1.0))
p.show()

In [ ]:
from proteusPy.Disulfide import DisulfideList, Disulfide, DisulfideLoader

# make some empty disulfides
ss1 = Disulfide('ss1')
ss2 = Disulfide('ss2')

# make a DisulfideList containing ss1, named 'tmp'
sslist = DisulfideList([ss1], 'tmp')
sslist.append(ss2)

# load the PDB Disulfide database
PDB_SS = None
PDB_SS = DisulfideLoader(verbose=True, modeldir=MODELS)

# extract a disulfide with typical index
ss1 = PDB_SS[0]
print(f'{ss1.pprint_all()}')

# grab a subset via slicing
subset = DisulfideList(PDB_SS[0:10],'subset')